# Work Flow
1. <a href="#1.-Problem-Description">Problem Description </a>
2. <a href="#2.-Data-Processing">Data Processing </a>
   * <a href="#Cleaning-Data">Cleaning Data</a>
   * <a href="#Vectorization">Vectorization</a><br>
3. <a href="#3.-Training">Data Processing </a>
   * <a href="#Naive-Bayes">Naive Bayes</a>
   * <a href="#LogisticRegression">LogisticRegression</a>$$$$
<a href="#Submission">Submission </a>


# 1. Problem Description
  Một vấn đề tồn tại đối với bất kỳ trang web lớn nào hiện nay là làm thế nào để xử lý nội dung độc hại và gây chia rẽ. Quora muốn giải quyết vấn đề này trực tiếp để giữ cho nền tảng của họ trở thành một nơi mà người dùng có thể cảm thấy an toàn khi chia sẻ kiến thức của họ với thế giới.$$$$
  Yêu cầu của bài toán là phân loại câu hỏi có phải là câu hỏi toxic hay không
       => **đây là bài toán classifier**
* Dữ liệu đầu vào sẽ là các câu hỏi (dữ liệu dạng text)
* Đầu ra là 0:1

>**Để đánh giá bài toán, ở đây ta dùng f1_score**
![](https://www.digital-mr.com/media/cache/5e/b4/5eb4dbc50024c306e5f707736fd79c1e.png) 
$$$$**Precision** được định nghĩa là tỉ lệ số điểm Positive mô hình dự đoán đúng trên tổng số điểm mô hình dự đoán là Positive
$$precision=\frac{TP}{TP + FP}$$
**Recall** được định nghĩa là tỉ lệ số điểm Positive mô hình dự đoán đúng trên tổng số điểm thật sự là Positive (hay tổng số điểm được gán nhãn là Positive ban đầu)
$$recall=\frac{TP}{TP + FN}$$
**F1-score**<br>
Tuy nhiên, chỉ có Precision hay chỉ có Recall thì không đánh giá được chất lượng mô hình.
>* Chỉ dùng Precision, mô hình chỉ đưa ra dự đoán cho một điểm mà nó chắc chắn nhất. Khi đó Precision = 1, tuy nhiên ta không thể nói là mô hình này tốt.
>* Chỉ dùng Recall, nếu mô hình dự đoán tất cả các điểm đều là positive. Khi đó Recall = 1, tuy nhiên ta cũng không thể nói đây là mô hình tốt.<br>
Khi đó F1-score được sử dụng. F1-score là trung bình điều hòa (harmonic mean) của precision và recall (giả sử hai đại lượng này khác 0). F1-score được tinh theo công thức:
$$\frac{1}{\frac{1}{precision} + \frac{1}{recall}}$$

**Thêm một số thư viện cần thiết**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

**Load dữ liệu vào data frame**

In [ ]:
# Load dũ liệu vào dataframe
train_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
train_df.info()

**Dữ liệu:**
* Tập dữ liệu train gồm 13006122 hàng x 3 cột
* 3 cột gồm 2 cột dữ liệu văn bản và 1 cột dữ liệu số
* Cột pid chứa id của câu hỏi
* Cột questions_text chứa nội dung câu hỏi
* Cột target có 2 giá trị (0 nếu là câu hỏi insincere, 1 nếu là câu hỏi sincere) 

In [ ]:
print('5 hàng dữ liệu đầu tiên của tập train')
train_df.head()

In [ ]:
print('5 hàng dữ liệu đầu tiên của tập test')
train_df.head()

**Nhận xét:** Ta thấy các câu hỏi trong cột question_text giữa tập train và tập test là khá tương đồng

In [ ]:
plt.figure(figsize=(10,9))
sns.countplot(x='target', data=train_df)
print('Sincere questions: ', train_df[train_df['target'] == 0].shape[0])
print('Insincere questions: ', train_df[train_df['target'] == 1].shape[0])
print('Total questions: ', train_df.shape[0])

In [ ]:
print('Phần trăm số câu hỏi hữu ích: {}%'.format(100 - round(train_df['target'].mean()*100, 2)))
print('\nPhần trăm số câu hỏi độc hại: {}%'.format(round(train_df['target'].mean()*100, 2)))

  **Nhận xét:**
* Số lượng câu hỏi sincere chiếm 93,81% lớn hơn rất nhiều so với số lượng câu hỏi insincere 
* Dữ liệu đang bị mất cân bằng nghiêm trọng
> Sử dụng thước đo đánh giá mô hình là độ chính xác (accuracy) sẽ không phù hợp. <br>
Việc tập dữ liệu bị mất cân bằng nghiêm trọng có thể dẫn tới sự dự báo kém chính xác của nhóm thiểu số (insincere question) 

 
 **Hướng giải quyết:**
* Chia lại tập dữ liệu theo tỉ lệ 4:1

# Phân tích dữ liệu bằng Word Cloud
* Word Cloud là một kỹ thuật trực quan hóa dữ liệu được sử dụng để biểu diễn dữ liệu văn bản, trong đó kích thước của mỗi từ cho biết tần suất hoặc tầm quan trọng của nó

In [ ]:
from wordcloud import WordCloud
print("Word cloud thể  hiện các câu hỏi sincere: ")
sincere_wordcloud = WordCloud(width=700, height=500, background_color='black', min_font_size=10).generate(str(train_df[train_df["target"] == 0]["question_text"]))
plt.figure(figsize=(10,9), facecolor=None)
plt.imshow(sincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
print("Word cloud thể  hiện các câu hỏi insincere: ")
insincere_wordcloud = WordCloud(width=700, height=500, background_color='black', min_font_size=10).generate(str(train_df[train_df["target"] == 1]["question_text"]))
plt.figure(figsize=(10,9), facecolor=None)
plt.imshow(insincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

# 2. Data Processing

* Như ta đã phân tích ở phần trước, tập dữ liệu đang bị mất cân bằng. Ta cần phải chia lại tập train sao cho cân bằng hơn
* Ở đây ta chia lại theo tỉ lệ 4:1 (4 hữu ích : 1 toxic)

In [ ]:
from sklearn.utils import resample
# Chia lại tập dữ liệu
sincere = train_df[train_df.target == 0]
insincere = train_df[train_df.target == 1]
df_train_sampled = pd.concat([resample(sincere,replace = True,n_samples = len(insincere)*4), insincere])


# Cleaning Data

Sau khi đánh giá ta thấy dữ liệu còn khá phức tạp và nhiều nhiễu. Để đơn giản hoá dữ liệu ta có thể thực hiện một số bước sau:
* Chuyển các từ in hoa thành in thường
* Chuyển các từ rút gọn về dạng hoàn chỉnh
* Các từ dừng (a, an, the...) xuất hiện rất nhiều trong câu hỏi nhưng không liên quan đến nội dung câu hỏi nên cần được loại bỏ
* Các số và ký tự đặc biệt cũng tương tự như từ dừng không ảnh hưởng đến ý nghĩa câu hỏi
* Chuyển các từ về dạng từ gốc

In [ ]:
import re
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

wordnet_lemmatizer = WordNetLemmatizer()
import re, string
tokenizer = ToktokTokenizer()
stemmer = SnowballStemmer('english')
stopword_list = nltk.corpus.stopwords.words('english')


In [ ]:
def clean_text(text):

    # Xoá HTML Tags
    text = re.sub(re.compile('<.*?>'), '', text)

    # Xoá  [\], ['], ["]
    text = re.sub(r'\\', '', text)
    text = re.sub(r'\"', '', text)
    text = re.sub(r'\'', '', text)

    # Xoá chữ số
    text = re.sub('[0-9]{5,}','#####', text);
    text = re.sub('[0-9]{4,}','####', text);
    text = re.sub('[0-9]{3,}','###', text);
    text = re.sub('[0-9]{2,}','##', text);

    # Xoá ký tự la mã
    roman = re.compile(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$');
    text = roman.sub(r'', text);

    # Chuyển các ký tự về dạng in thường
    text = text.strip().lower()

    # Xoá các ký tự đặc biệt
    filters = '!"\'#$%@&*()+_-;:<=>.?{}|`\\^\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    #token 
    #tokens = tokenizer.tokenize(text)
    #tokens = [token.strip() for token in tokens]
   # tokens = [token for token in tokens if token not in stop_words]
   # tokens = [stemmer.stem(token) for token in tokens]
    #tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    
    return text

In [ ]:
df_train_sampled['preprocessed_questions'] = df_train_sampled['question_text'].apply(clean_text)

In [ ]:
print('Cột preprocessed_ question chứa các câu hỏi đã được làm s')
df_train_sampled.head()

# Vectorization

**Vectorization:** Trước khi đưa vào huấn luyện ta cần chuyển dữ liệu từ dạng vô hướng sang dạng vector<br>
**Mô hình túi từ (bag-of-words)** Mô hình bag-of-words (BOW) là cách biểu diễn biến văn bản tùy ý thành các vectơ có độ dài cố định bằng cách đếm số lần mỗi từ xuất hiện, không quan tâm đến ngữ pháp và thậm chí trật tự từ nhưng vẫn giữ tính đa dạng.
>Bước 1: Xác định túi từ vựng gồm: the, cat, sat, in, hat, with <br>
      Bước 2: Đếm số lần xuất hiện cảu các từ
![](https://miro.medium.com/max/1536/1*3IACMnNpwVlCl8kSTJocPA.png)  
* Mô hình túi từ thường được sử dụng trong các phương pháp phân loại tài liệu trong đó sự xuất hiện (tần suất) của mỗi từ được sử dụng như một đặc trưng để đào tạo máy phân loại
* **Ưu điểm:** Đơn giản, tính toán nhanh, dễ tính toán. Không quan tâm đến vị trí và ngữ cảnh<br>

**CountVectorize:** 
* được sử dụng để chuyển đổi một bộ sưu tập các tài liệu văn bản thành một vectơ có số lượng thuật ngữ / mã thông báo. Nó cũng cho phép xử lý trước dữ liệu văn bản trước khi tạo biểu diễn vectơ. Chức năng này làm cho nó trở thành một mô-đun biểu diễn tính năng rất linh hoạt cho văn bản


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

X = df_train_sampled['preprocessed_questions']
y = df_train_sampled.target
test = test_df['question_text']
X_train, X_test, y_train, y_test = train_test_split(df_train_sampled['preprocessed_questions'], df_train_sampled['target'], test_size=0.3)

* Chia tập dữ liệu ra thành 2 phần train, test với test_size = 0.3

In [ ]:
cnt_vectorizer = CountVectorizer()
#Fitting count vectorizer to both training and test sets (semi-supervised learning)
cnt_vectorizer.fit(list(X_train) + list(X_test))
X_train = cnt_vectorizer.transform(X_train) 
X_test = cnt_vectorizer.transform(X_test)

In [ ]:
df_train_sampled["preprocessed_questions"].head()


# 3. Trainning

# Naive Bayes
* **Naive Bayes Classification** (NBC) là một thuật toán dựa trên định lý Bayes về lý thuyết xác suất  để đưa ra các phán đoán cũng như phân loại dữ liệu dựa trên các dữ liệu được quan sát và thống kê. Naive Bayes Classification là một trong những thuật toán được ứng dụng rất nhiều trong các lĩnh vực Machine learning dùng để đưa các dự đoán chính xác nhất dự trên một tập dữ liệu đã được thu thập, vì nó khá dễ hiểu và độ chính xác cao. Nó thuộc vào nhóm Supervised Machine Learning Algorithms (thuật toán học có hướng dẫn), tức là máy học từ các ví dụ từ các mẫu dữ liệu đã có.$$$$
 1. Gọi D là tập dữ liệu huấn luyện, trong đó mỗi phần tử dữ liệu X được biểu diễn bằng một vector chứa n giá trị thuộc tính A1, A2,...,An = {x1,x2,...,xn}
 2. Giả sử có m lớp C1, C2,..,Cm. Cho một phần tử dữ liệu X, bộ phân lớp sẽ gán nhãn cho X là lớp có xác suất hậu nghiệm lớn nhất. Cụ thể, bộ phân lớp Bayes sẽ dự đoán X thuộc vào lớp Ci nếu và chỉ nếu:
$$P(Ci|X) > P(Cj|X) (1<= i, j <=m, i != j)$$
 3. Giá trị này sẽ tính dựa trên định lý Bayes.
Để tìm xác suất lớn nhất, ta nhận thấy các giá trị P(X) là giống nhau với mọi lớp nên không cần tính. Do đó ta chỉ cần tìm giá trị lớn nhất của P(X|Ci) * P(Ci). Chú ý rằng P(Ci) được ước lượng bằng |Di|/|D|, trong đó Di là tập các phần tử dữ liệu thuộc lớp Ci. Nếu xác suất tiền nghiệm P(Ci) cũng không xác định được thì ta coi chúng bằng nhau P(C1) = P(C2) = ... = P(Cm), khi đó ta chỉ cần tìm giá trị P(X|Ci) lớn nhất.<br>
 4. Khi số lượng các thuộc tính mô tả dữ liệu là lớn thì chi phí tính toàn P(X|Ci) là rất lớn, dó đó có thể giảm độ phức tạp của thuật toán Naive Bayes giả thiết các thuộc tính độc lập nhau. Khi đó ta có thể tính:
$$P(X|Ci) = P(x1|Ci)...P(xn|Ci)$$


In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
from sklearn.naive_bayes import MultinomialNB
MulNB = MultinomialNB()
MulNB.fit(X_train,y_train)
test_predictions = MulNB.predict(X_test)
test_acc = accuracy_score(y_test, test_predictions) 
test_f1 = f1_score(y_test, test_predictions)
print('Precision: %.3f' % precision_score(y_test, test_predictions))
print('Recall: %.3f' % recall_score(y_test,test_predictions))
print(f"Testing accuracy:  {test_acc:.2%}, F1: {test_f1:.4f}")
y_pred_proba = MulNB.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

Đánh giá:
* Tốc độ chạy mô hình khá nhanh  do mô hình đơn giản
* Điểm accuracy khá là cao do tập dữ liệu mất cân bằng
* F1 score đạt được ở mức trung bình

# LogisticRegression

**Logistic Regression** là 1 thuật toán phân loại được dùng để gán các đối tượng cho 1 tập hợp giá trị rời rạc (như 0, 1, 2, ...). Một ví dụ điển hình là phân loại Email, gồm có email công việc, email gia đình, email spam, ...
* Cũng giống như các thuật toán Linear (tìm và tối ưu w) với công thức chung: $$y = f(w^{T}x),(w \in [w_{0}, w_{1},... w_{n}])$$
* Đầu ra dự đoán của logistic regression thường được viết chung dưới dạng:$$y = \theta (w^{T}x)$$
Trong đó θ được gọi là logistic function.

**Sigmoid function**<br>
 ![]( https://nickmccullum.com/images/python-deep-learning/deep-learning-activation-functions/sigmoid-function.png)

* Như vậy sau khi tính được phần tuyến tính là $w^{T}x$, phần tuyến tính này sẽ được đưa vào hàm sigmoid($\sigma()$). Hàm $\sigma()$ sẽ trả về xác suất mà câu hỏi là hữu ích ($y = 0$) hay độc hại ($y = 1$) với: $$P(y_{1}|x_{i};w) = \sigma(w^{T}x); P(y_{0}|x_{i};w) = 1 - \sigma(w^{T}x)$$
* Bài toán quay trở về với việc **tối ưu hàm mất mát L** với mục tiêu làm cho $P(y|x_{i}) = \sigma(w^{T}x) = \widehat{y}$ càng ngày càng lớn (tiến gần tới 1): $$L = -log(P(y|x_{i}))$$ $$<=> \frac{\partial L}{\partial w} = (\widehat{y} - y)x = (\sigma(w^{T}x) - y)x$$ $$=> w = w + \lambda(\widehat{y} - y)x$$
* Với $\lambda$ là hệ học learning_rate (là một con số nhỏ thường nằm trong khoảng từ 0.01 đến 0.0001). <br>

In [ ]:
from sklearn.linear_model import LogisticRegression
# Fitting a simple Logistic Regression
logisticRegression = LogisticRegression(max_iter = 5000)
logisticRegression.fit(X_train,y_train)
test_predictions = logisticRegression.predict(X_test)
test_acc = accuracy_score(y_test, test_predictions) 
test_f1 = f1_score(y_test, test_predictions) 
print(f"Testing accuracy:  {test_acc:.2%}, F1: {test_f1:.4f}")
print('Precision: %.3f' % precision_score(y_test, test_predictions))
print('Recall: %.3f' % recall_score(y_test,test_predictions))
y_pred_proba = logisticRegression.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()


Nhận xét:
* Ở đây ta nhận được kết quả tốt hơn so với mô hình Naice Bayes
* Thuật toán cũng chạy chậm hơn

# Submission

In [ ]:
x_val = cnt_vectorizer.transform(test_df['question_text'])
validation_predictions = logisticRegression.predict(x_val)
submission = pd.DataFrame({'qid':test_df['qid'], 'prediction':validation_predictions })
submission.to_csv('submission.csv', index=False)
submission